### Secure tool calling with Arcade.dev

This notebook is going to close one of the biggest gaps between demo and production agents: Secure tool-calling

When your agents work well in your computer, they are excellent personal assistants, but scaling that up to many users is not easy, as the security assumptions from a local deployment do not apply to agents at scale. Personal Access Tokens simply won't cut it for multiple users. Even if you encapsulate all of the functionality in a remote MCP server, tool-level auth will require you to implement the auth flow for all the providers that your agent relies on.

Arcade solves this by providing a unified platform for agentic tool calling and execution. It will handle the auth flow for you offering a secure multi-user solution for your agents.

In this notebook we will learn how to use Arcade and LangGraph to :-

- Build agents
- Give tools that can interact with
    - GMail
    - Slack
    - Notion
- Implement safety guardrails when calling specific tools (Human-in-the-Loop)

## Development Environment Setup

Before implementing our multi-user agent system, we need to establish a proper development environment with the necessary dependencies. The following installation includes LangGraph for agent orchestration, LangChain-Arcade for tool integration, and the core LangChain library with OpenAI support.

In [1]:
# !pip3 install langgraph langchain-arcade langchain langchain-openai
# Install required packages
!pip3 install langchain langchain-openai # langgraph langchain-arcade python-dotenv

  Using cached langchain-1.0.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_openai-1.0.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached langchain_core-1.0.4-py3-none-any.whl.metadata (3.5 kB)
  Using cached langgraph-1.0.3-py3-none-any.whl.metadata (7.8 kB)
  Using cached pydantic-2.12.4-py3-none-any.whl.metadata (89 kB)
  Using cached tiktoken-0.12.0-cp312-cp312-macosx_10_13_x86_64.whl.metadata (6.7 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.4.42-py3-none-any.whl.metadata (14 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-macosx_10_13_x86_64.whl.metadata (2.4 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached langgraph_checkpoint-3.0.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached xxhash-3.6.0-cp312-cp312-macosx_10_13_x86_64.whl.metada

## API Key Configuration

Our tutorial requires two essential API keys for operation. You will need an [OpenAI API](https://platform.openai.com/signup) key, as well as an [Arcade API](https://api.arcade.dev/signup?utm_source=github&utm_medium=notebook&utm_campaign=nir_diamant&utm_content=tutorial) key for this tutorial. Both services offer straightforward registration processes, with Arcade specifically designed to simplify the integration of external tools into AI applications.

In [4]:
!pip3 install python-dotenv

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file (create this with your API key)
load_dotenv()

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Set Arcade API key
os.environ["ARCADE_API_KEY"] = os.getenv('ARCADE_API_KEY')

## User Identity Configuration

The Arcade platform requires user identification to properly manage tool authorizations and maintain security boundaries between different users. This identifier must correspond to the email address used during Arcade account creation, ensuring that tool permissions and OAuth tokens are correctly associated with the appropriate user account.

In [3]:
# Set ARCADE user id which is the email address used furing Arcade account creation
os.environ["ARCADE_USER_ID"] = os.getenv('ARCADE_USER_ID')


# Simple Conversational Agent

We begin our journey by implementing a basic conversational agent that demonstrates core LangGraph functionality without external tool dependencies. This foundational agent provides conversational capabilities with short-term memory, allowing it to maintain context throughout a conversation while establishing the architectural patterns we'll extend throughout this tutorial.

## Core Agent Implementation

The following implementation creates a ReAct-style agent using [LangGraph and Arcade](https://docs.arcade.dev/home/langchain/use-arcade-tools#create-a-react-style-agent?utm_source=github&utm_medium=notebook&utm_campaign=nir_diamant&utm_content=tutorial). We configure it with conversation memory through a MemorySaver checkpointer, enabling the agent to remember previous interactions within the same conversation thread. The agent receives a clear prompt defining its helpful and concise personality, along with instructions for handling unclear requests.

In [6]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
import uuid

# create a checkpointer to persist the graph's state
checkpointer = MemorySaver()

agent_a = create_agent(
    model="openai:gpt-5-mini",
    system_prompt="You are a helpful assistant that can help with everyday tasks."
           " If the user's request is confusing you must ask them to clarify"
           " their intent, and fulfill the instruction to the best of your"
           " ability. Be concise and friendly at all times.",
    tools=[], # no tools for now!
    checkpointer=checkpointer
)

## Agent Interaction Utility

To facilitate consistent interaction with our agents throughout this tutorial, we implement a utility function that streams agent responses and displays them in a readable format. This function processes the graph's streaming output and presents the latest message from each interaction cycle, providing immediate feedback during agent conversations.

In [7]:
from langgraph.graph.state import CompiledStateGraph

def run_graph(graph: CompiledStateGraph, config, input):
    
    for event in graph.stream(input, config=config, stream_mode="values"):
        if "messages" in event:
            event["messages"][-1].pretty_print()

## Interactive Chat Interface

The following implementation provides a complete interactive chat interface for testing our basic agent. The system generates a unique conversation thread identifier for each session, enabling memory persistence across multiple exchanges within the same conversation. Users can engage naturally with the agent and terminate the session by typing "exit".

In [8]:
# the configuration helps LangGraph keep track of conversations and interrups
# While it's not needed for this agent. The agent will remember different
# conversations based on the thread_id. This code generates a random id every
# time you run the cell, but you can hardcode the thread_id if you want to
# test the memory.
config = {
    "configurable": {
        "thread_id": uuid.uuid4()
    }
}
while True:
    user_input = input("👤: ")
    # let's use "exit" as a safe way to break the infinite loop
    if user_input.lower() == "exit":
        break

    user_message = {"messages": [HumanMessage(content=user_input)]}
    run_graph(agent_a, config, user_message)



================================ Human Message =================================

Which actor has won the most oscars?
================================== Ai Message ==================================

If you mean acting Oscars: Katharine Hepburn holds the record — she won four Academy Awards for Best Actress.

For other common interpretations:
- Most Oscars for a male actor: Daniel Day‑Lewis (three Best Actor wins).
- Most Oscars overall (any category): Walt Disney, with 22 competitive plus 4 honorary Oscars.


## Testing Agent Limitations / Demonstrating Authentication Requirements

The following test illustrates the agent's complete inability to access private, authenticated data sources. When asked to summarize personal emails, the agent cannot proceed without proper authentication mechanisms and authorized access to external services. This limitation highlights the critical need for secure tool integration in production agent systems.

To understand the boundaries of our basic agent, we'll test it with requests that require external data access. The following test demonstrates the agent's inability to provide current date information, as most language models lack real-time data access and may provide outdated or inaccurate temporal information.

In [9]:
config = {
    "configurable": {
        "thread_id": uuid.uuid4()
    }
}
print(f'thread_id = {config["configurable"]["thread_id"]}')

prompt = "summarize my latest 3 emails please"
user_message = {"messages": [HumanMessage(content=prompt)]}
run_graph(agent_a, config, user_message)

thread_id = 4eefd1e8-8388-456b-9255-ca74aff3b824
================================ Human Message =================================

summarize my latest 3 emails please
================================== Ai Message ==================================

I don’t have access to your email inbox. Please paste the three emails (or their subject, sender and body) here and I’ll summarize them.

A couple quick questions so I format the summaries the way you want:
- Do you want 1-line summaries, short bullets, or a longer paragraph each?
- Include action items/deadlines and suggested replies? (yes/no)
- Do you want sender names and subject lines shown in the summary?

Tip: paste each email with a clear separator, e.g.
---
Subject: [subject]
From: [name/email]
Body: [email text]
---

Don’t include passwords, credit-card numbers, or other highly sensitive info.


# Tool Integration with Secure Authentication

Having established our basic agent architecture, we now address the core challenge of enabling secure access to external services. This section demonstrates how Arcade.dev solves the complex problem of tool-level authentication, providing a streamlined approach to OAuth integration that scales across multiple users and services.

## Arcade Client Initialization

We begin by establishing connections to the Arcade platform through both the core client and the LangChain integration layer. The ToolManager serves as our primary interface for configuring and authorizing tools, while the Arcade client handles the underlying authentication infrastructure.

In [11]:
!pip3 install langchain-arcade 

  Using cached langchain_arcade-1.4.4-py3-none-any.whl.metadata (6.5 kB)
  Using cached arcadepy-1.10.0-py3-none-any.whl.metadata (14 kB)
  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_arcade-1.4.4-py3-none-any.whl (11 kB)
Using cached arcadepy-1.10.0-py3-none-any.whl (118 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.4
    Uninstalling langchain-core-1.0.4:
      Successfully uninstalled langchain-core-1.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.4 requires langchain-core>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-openai 1.0.2 requires langchain-core<2.0.0,>=1.0.2, but you have langchain-core 0.3.79 which is incompatible.
langchain 1.0.5 req

In [12]:
from langchain_arcade import ToolManager
from arcadepy import Arcade

arcade_client = Arcade(api_key=os.getenv("ARCADE_API_KEY"))
manager = ToolManager(client=arcade_client)

## Gmail Tool Configuration

Our first tool integration focuses on Gmail access, specifically the email listing capability that our basic agent could not provide. The Gmail_ListEmails tool enables our agent to retrieve and analyze email data, but requires proper user authorization before it can access private email accounts.

In [13]:
gmail_tool = manager.init_tools(tools=["Gmail_ListEmails"])[0]

## Authorization Utility Function

To streamline the authorization process throughout this tutorial, we implement a reusable function that handles OAuth flow initiation and completion. For reading our email, however, we need to give our app permissions to read it in a secure way. Arcade lets us do this easily by [handling the OAuth2 for us](https://docs.arcade.dev/home/auth/how-arcade-helps?utm_source=github&utm_medium=notebook&utm_campaign=nir_diamant&utm_content=tutorial). This function checks the current authorization status for a specific tool and user combination, initiating the OAuth process when necessary and waiting for user completion of the authorization flow.

In [14]:
def authorize_tool(tool_name, user_id, manager):
    # This line will check if this user is authorized to use the
    # tool, and return a response that we can use if the user
    # did not authorize the tool yet.
    auth_response = manager.authorize(
        tool_name=tool_name,
        user_id=user_id
    )
    if auth_response.status != "completed":
        print(f"The app wants to use the {tool_name} tool.\n"
              f"Please click this url to authorize it {auth_response.url}")
        # wait until the user authorizes
        manager.wait_for_auth(auth_response.id)


## Gmail Authorization Process

The following cell initiates the authorization process for Gmail access. If the user has not previously granted permissions, Arcade will provide an OAuth URL for completing the authorization. Once authorized, the permission persists for future sessions, eliminating the need for repeated authorization flows.

In [15]:
authorize_tool(gmail_tool.name, os.getenv("ARCADE_USER_ID"), manager)

## Enhanced Agent with Gmail Capabilities

With Gmail authorization complete, we can now create an enhanced agent that incorporates email access capabilities. This agent retains all the conversational abilities of our basic implementation while adding the power to interact with authenticated email services. Notice the updated prompt that explicitly mentions Gmail capabilities and the inclusion of the user_id in the configuration for tool execution.

In [17]:
# define a new agent, this time with access to our tool!
agent_b = create_agent(
    model="openai:gpt-5",
    system_prompt="You are a helpful assistant that can help with everyday tasks."
           " If the user's request is confusing you must ask them to clarify"
           " their intent, and fulfill the instruction to the best of your"
           " ability. Be concise and friendly at all times."
           # It's useful to let the agent know about the tools it has at its disposal.
           " Use the Gmail tools that you have to address requests about emails.",
    tools=[gmail_tool], # we pass the tool we previously authorized.
    checkpointer=checkpointer
)

config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
        "user_id": os.getenv("ARCADE_USER_ID") # When using Arcade tools, we must provide the user_id on the LangGraph config, so Arcade can execute the tool invoked by the agent.
    }
}
print(f'thread_id = {config["configurable"]["thread_id"]}')
print(f'user_id = {config["configurable"]["user_id"]}')

# we're using the same prompt we use before, but we're swapping the agent
prompt = "summarize my latest 3 emails please"
user_message = {"messages": [HumanMessage(content=prompt)]}
run_graph(agent_b, config, user_message)


thread_id = b414ac34-e3e7-4225-a0d9-41d8fca1a5a6
user_id = abdullahmakhdoom1998@gmail.com
================================ Human Message =================================

summarize my latest 3 emails please
================================== Ai Message ==================================
Tool Calls:
  Gmail_ListEmails (call_yB9n1vOcVIpXWH4ePvo7I5Ps)
 Call ID: call_yB9n1vOcVIpXWH4ePvo7I5Ps
  Args:
    n_emails: 3
================================= Tool Message =================================
Name: Gmail_ListEmails

{"emails": [{"body": "To view this content open the following URL in your browser: https://www.pinterest.com/email/click/?user_id=NjIyMjAwNjQyMTc3MzM5NjE4&od=dD03ZGUwODEwMzgxN2I0ODg4OWJjYjI5YTNjNGJjZmI3ZSZjPUhPTUVGRUVEX0RJR0VTVF9QSU5TJnM9N2RlMDgxMDM4MTdiNDg4ODliY2IyOWEzYzRiY2ZiN2Umbj0wN2ZmODU5NjdiZDE0MTQxODZmNGNiOGQzNDM4NmI3Yw%3D%3D⌖=https%3A%2F%2Fwww.pinterest.com%2Fsecure%2Fautologin%2F%3Fuser_id%3DNjIyMjAwNjQyMTc3MzM5NjE4%26od%3DbMydcndybDelDZXQ0q18I6R%252BBWQ%252B%252FfW

# Multi-Service Tool Integration

Building upon our successful Gmail integration, we now expand our agent's capabilities to include multiple external services. This section demonstrates how to efficiently manage authentication across multiple providers while maintaining security and user experience standards.

## Batch Authorization Utility

Managing multiple tool authorizations individually becomes cumbersome as our agent's capabilities expand. This requires [initializing multiple tools](https://docs.arcade.dev/home/faq#can-i-authenticate-multiple-tools-at-once?utm_source=github&utm_medium=notebook&utm_campaign=nir_diamant&utm_content=tutorial) for the agent, and authenticating the scope of each tool. The following function streamlines this process by grouping authorization scopes by provider, minimizing the number of OAuth flows users must complete while ensuring comprehensive tool access.

In [18]:
def authorize_tools(tools, user_id, client):

    # This will map all the providers to the specific scopes they need
    provider_to_scopes = {}
    for tool in tools:
        provider = tool.requirements.authorization.provider_id
        if provider not in provider_to_scopes:
            provider_to_scopes[provider] = set()

        if tool.requirements.authorization.oauth2.scopes:
            provider_to_scopes[provider] |= set(tool.requirements.authorization.oauth2.scopes)

    # Each provider will handle its own scopes, we iterate and present the
    # auth URL for all providers that need it
    for provider, scopes in provider_to_scopes.items():
        # start auth
        auth_response = client.auth.start(
            user_id=user_id,
            scopes=list(scopes),
            provider=provider
        )

        # show the url to the user if needed
        if auth_response.status != "completed":
            print(f"🔗 Please click here to authorize: {auth_response.url}")
            print(f"⏳ Waiting for authorization completion...")

            # Wait for the authorization to complete with timeout
            client.auth.wait_for_completion(auth_response),


## Comprehensive Tool Suite Configuration

We now expand our agent's capabilities by incorporating tools for email sending, Slack communication, and Notion content management. This configuration provides our agent with the ability to not only read information from various services but also to create and send content, enabling more sophisticated workflow automation.

In [19]:
# add a single tool
manager.add_tool("Gmail.SendEmail")
# add an entire toolkit (a collection of tools)
manager.add_toolkit("Slack")
manager.add_toolkit("NotionToolkit")

## Multi-Service Authorization

The following cell executes the authorization process for all configured tools simultaneously. This efficient approach minimizes user interaction while establishing the necessary permissions for Gmail, Slack, and Notion access. The batch authorization system automatically groups scopes by provider to present the minimum number of authorization flows.

In [20]:
authorize_tools(
    tools=manager.definitions,
    user_id=os.getenv("ARCADE_USER_ID"),
    client=arcade_client
)

## Multi-Service Agent Implementation

With comprehensive tool authorization complete, we create our most capable agent yet. This implementation leverages the ToolManager's LangChain conversion functionality to provide seamless integration between Arcade's tool definitions and LangGraph's execution framework. The enhanced prompt guides the agent in selecting appropriate tools for different types of requests.

In [21]:
# define a new agent, this time with access to our tool!
agent_c = create_agent(
    model="openai:gpt-5",
    system_prompt="You are a helpful assistant that can help with everyday tasks."
           " If the user's request is confusing you must ask them to clarify"
           " their intent, and fulfill the instruction to the best of your"
           " ability. Be concise and friendly at all times."
           # It's useful to let the agent know about the tools it has at its disposal.
           " Use the Gmail tools to address requests about reading or sending emails."
           " Use the Slack tools to address requests about interactions with users and channels in Slack."
           " Use the Notion tools to address requests about managing content in Notion Pages."
           " In general, when possible, use the most relevant tool for the job.",
    tools=manager.to_langchain(),
    checkpointer=checkpointer
)



## Complex Multi-Service Task Execution

This demonstration showcases our agent's ability to orchestrate complex workflows across multiple services. The request requires the agent to analyze email data, retrieve Slack communications, and explore Notion workspace structure, demonstrating sophisticated tool selection and execution coordination.

In [24]:
config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
        "user_id": os.getenv("ARCADE_USER_ID") # When using Arcade tools, we must provide the user_id on the LangGraph config, so Arcade can execute the tool invoked by the agent.
    }
}
print(f'thread_id = {config["configurable"]["thread_id"]}')

# we're using the same prompt we use before, but we're swapping the agent
prompt = """
    Summarize my latest 3  emails. If there is a confirmation code in these 3 last emails, send it to me on Slack. 
    Also create a new page "To Do - 2" within "To do list" page in Notion and add this code to it.
    Please do share the link of the newly created notion page.
    """
user_message = {"messages": [HumanMessage(content=prompt)]}
run_graph(agent_c, config, user_message)

thread_id = c6b8cadd-1360-4434-9dcd-7c03f5611a96
================================ Human Message =================================


    Summarize my latest 3  emails. If there is a confirmation code in these 3 last emails, send it to me on Slack. 
    Also create a new page "To Do - 2" within "To do list" page in Notion and add this code to it.
    Please do share the link of the newly created notion page.
    
================================== Ai Message ==================================
Tool Calls:
  Gmail_ListEmails (call_i7BNYyx6qHB5p16MjIznum7m)
 Call ID: call_i7BNYyx6qHB5p16MjIznum7m
  Args:
    n_emails: 3
  Slack_WhoAmI (call_cuJ5dkPOywu0T2JOBVQhebYJ)
 Call ID: call_cuJ5dkPOywu0T2JOBVQhebYJ
  Args:
================================= Tool Message =================================
Name: Slack_WhoAmI

{"email": "abdullahmakhdoom1998@gmail.com", "first_name": "Abdullah", "last_name": "Makhdoom", "profile_picture_url": "https://avatars.slack-edge.com/2022-09-28/4140150820277_adc55a

# Human-in-the-Loop Safety Implementation

While our multi-service agent demonstrates impressive capabilities, production systems require robust safety mechanisms to prevent unintended actions. This section implements human-in-the-loop controls for sensitive operations, ensuring that potentially harmful or irreversible actions require explicit user approval before execution.

## Identifying Sensitive Operations

Before implementing safety controls, we must identify which tools require human oversight. The following examination of available tools helps us categorize operations based on their potential impact and irreversibility.

In [25]:
for tool_name, _ in manager:
    print(tool_name)

Gmail_ListEmails
Gmail_SendEmail
Slack_GetConversationMetadata
Slack_GetMessages
Slack_GetUsersInConversation
Slack_GetUsersInfo
Slack_ListConversations
Slack_ListUsers
Slack_SendMessage
Slack_WhoAmI
NotionToolkit_AppendContentToEndOfPage
NotionToolkit_CreatePage
NotionToolkit_GetObjectMetadata
NotionToolkit_GetPageContentById
NotionToolkit_GetPageContentByTitle
NotionToolkit_GetWorkspaceStructure
NotionToolkit_SearchByTitle
NotionToolkit_WhoAmI


## Sensitive Tool Classification

Based on potential impact analysis, we identify tools that could cause unintended consequences if executed with incorrect parameters. These tools typically involve creating, sending, or modifying data rather than simply retrieving information. The classification focuses on operations that have external effects or could compromise user privacy or system integrity.

In [26]:
tools_to_protect = [
    "Gmail_SendEmail",
    "Slack_SendMessage",
    "NotionToolkit_AppendContentToEndOfPage",
    "NotionToolkit_CreatePage",
    "NotionToolkit_CreatePage"
]

## Human-in-the-Loop Tool Wrapper

The following implementation creates a wrapper function that transforms regular tools into human-supervised versions. This wrapper intercepts tool execution requests, presents the planned action to the user for approval, and only proceeds with execution upon receiving explicit consent. The implementation leverages LangGraph's interrupt mechanism to pause execution pending user input.

In [27]:
from typing import Callable, Any
from langchain_core.tools import tool, BaseTool
from langgraph.types import interrupt, Command
from langchain_core.runnables import RunnableConfig
import pprint


def add_human_in_the_loop(
    target_tool: Callable | BaseTool,
) -> BaseTool:
    """Wrap a tool to support human-in-the-loop review."""
    if not isinstance(target_tool, BaseTool):
        target_tool = tool(target_tool)

    @tool(
        target_tool.name,
        description=target_tool.description,
        args_schema=target_tool.args_schema
    )
    def call_tool_with_interrupt(config: RunnableConfig, **tool_input):

        arguments = pprint.pformat(tool_input, indent=4)
        response = interrupt(
            f"Do you allow the call to {target_tool.name} with arguments:\n"
            f"{arguments}"
        )

        # approve the tool call
        if response == "yes":
            tool_response = target_tool.invoke(tool_input, config)
        # deny tool call
        elif response == "no":
            tool_response = "The User did not allow the tool to run"
        else:
            raise ValueError(
                f"Unsupported interrupt response type: {response}"
            )

        return tool_response

    return call_tool_with_interrupt


## Selective Tool Protection Application

This implementation applies human-in-the-loop protection selectively, wrapping only the tools identified as sensitive while leaving read-only operations unchanged. This approach maintains agent efficiency for safe operations while ensuring appropriate oversight for potentially risky actions.

In [28]:
protected_tools = [
    add_human_in_the_loop(t)
    if t.name in tools_to_protect else t
    for t in manager.to_langchain()
]

## Interrupt Handling Utilities


LangGraph interrupts require specialized handling to resume execution after user input. The following utilities provide a user-friendly interface for approval decisions and automate the process of resuming agent execution with the user's response. The yes/no loop ensures clear decision-making while the interrupt handler manages the technical aspects of execution resumption.

In [29]:
def yes_no_loop(prompt: str) -> str:
    """
    Force the user to say yes or no
    """
    print(prompt)
    user_input = input("Your response [y/n]: ")
    while user_input.lower() not in ["y", "n"]:
        user_input = input("Your response (must be 'y' or 'n'): ")
    return "yes" if user_input.lower() == "y" else "no"


def handle_interrupts(graph: CompiledStateGraph, config):
    for interr in graph.get_state(config).interrupts:
        approved = yes_no_loop(interr.value)
        run_graph(graph, config, Command(resume=approved))


## Protected Agent Implementation

Our final agent implementation incorporates comprehensive safety controls while maintaining all the multi-service capabilities developed throughout this tutorial. This agent represents a production-ready system that balances functionality with security, ensuring that users maintain control over sensitive operations while benefiting from automated assistance for routine tasks.

In [30]:
# define a new agent, this time with access to our tool!
agent_hitl = create_agent(
    model="openai:gpt-5",
    system_prompt="You are a helpful assistant that can help with everyday tasks."
           " If the user's request is confusing you must ask them to clarify"
           " their intent, and fulfill the instruction to the best of your"
           " ability. Be concise and friendly at all times."
           # It's useful to let the agent know about the tools it has at its disposal.
           " Use the Gmail tools to address requests about reading or sending emails."
           " Use the Slack tools to address requests about interactions with users and channels in Slack."
           " Use the Notion tools to address requests about managing content in Notion Pages."
           " In general, when possible, use the most relevant tool for the job.",
    tools=protected_tools,
    checkpointer=checkpointer
)

## Safety Mechanism Demonstration

The following test demonstrates our safety system in action by attempting to send a potentially sensitive email. This scenario illustrates how the human-in-the-loop mechanism intercepts the action, presents the details for user review, and awaits explicit approval before proceeding with execution.

In [31]:
config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
        "user_id": os.getenv("ARCADE_USER_ID") # When using Arcade tools, we must provide the user_id on the LangGraph config, so Arcade can execute the tool invoked by the agent.
    }
}
print(f'thread_id = {config["configurable"]["thread_id"]}')

# we're using the same prompt we use before, but we're swapping the agent
prompt = 'Send an email with subject "confidential data" and body "this is top secret information" to my own email of abdullahmakhdoom1998@gmail.com. :} '
user_message = {"messages": [HumanMessage(content=prompt)]}
run_graph(agent_hitl, config, user_message)

thread_id = 4ec90dbb-7952-43fb-b743-0300735d2529
================================ Human Message =================================

Send an email with subject "confidential data" and body "this is top secret information" to my own email of abdullahmakhdoom1998@gmail.com. :} 
================================== Ai Message ==================================
Tool Calls:
  Gmail_SendEmail (call_wjuMBqKLKQJknlY8fFqo8yKi)
 Call ID: call_wjuMBqKLKQJknlY8fFqo8yKi
  Args:
    subject: confidential data
    body: this is top secret information
    recipient: abdullahmakhdoom1998@gmail.com


## Interrupt State Inspection

When our safety system activates, the agent execution pauses and enters an interrupt state. The following examination reveals the pending approval request, demonstrating how the system captures the intended action details and awaits user decision before proceeding.

In [32]:
agent_hitl.get_state(config).interrupts

(Interrupt(value="Do you allow the call to Gmail_SendEmail with arguments:\n{   'body': 'this is top secret information',\n    'recipient': 'abdullahmakhdoom1998@gmail.com',\n    'subject': 'confidential data'}", id='5815a69196a236deac4009d5d8f979f9'),)

## User Decision Processing

The following cell processes the pending interrupt, presenting the action details to the user and collecting their approval decision. This demonstration shows how users can review potentially sensitive actions and make informed decisions about whether to proceed with agent-proposed operations.

In [33]:
handle_interrupts(agent_hitl, config)

Do you allow the call to Gmail_SendEmail with arguments:
{   'body': 'this is top secret information',
    'recipient': 'abdullahmakhdoom1998@gmail.com',
    'subject': 'confidential data'}


================================== Ai Message ==================================
Tool Calls:
  Gmail_SendEmail (call_wjuMBqKLKQJknlY8fFqo8yKi)
 Call ID: call_wjuMBqKLKQJknlY8fFqo8yKi
  Args:
    subject: confidential data
    body: this is top secret information
    recipient: abdullahmakhdoom1998@gmail.com
================================= Tool Message =================================
Name: Gmail_SendEmail

{"body": "", "cc": "", "date": "", "from": "", "header_message_id": "", "history_id": "", "id": "19a82697ecaacc77", "in_reply_to": "", "label_ids": ["UNREAD", "SENT", "INBOX"], "references": "", "reply_to": "", "snippet": "", "subject": "", "thread_id": "19a82697ecaacc77", "to": "", "url": "https://mail.google.com/mail/u/0/#sent/19a82697ecaacc77"}
================================== Ai Message ==================================

Your email has been sent to abdullahmakhdoom1998@gmail.com with the subject "confidential data."


## Complete Interactive System

This final implementation provides a complete interactive system that combines all the capabilities developed throughout this tutorial. Users can engage in natural conversations with an agent that has access to multiple external services while maintaining safety through human-in-the-loop controls for sensitive operations. The system automatically handles authorization, tool execution, and safety approvals in a seamless user experience.

In [34]:
config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
        "user_id": os.getenv("ARCADE_USER_ID") # When using Arcade tools, we must provide the user_id on the LangGraph config, so Arcade can execute the tool invoked by the agent.
    }
}

while True:
    user_input = input("👤: ")
    # let's use "exit" as a safe way to break the infinite loop
    if user_input.lower() == "exit":
        break

    user_message = {"messages": [HumanMessage(content=user_input)]}

    run_graph(agent_hitl, config, user_message)

    print("Interruption : ", agent_hitl.get_state(config).interrupts)

    handle_interrupts(agent_hitl, config)

================================ Human Message =================================

Message me the subject and short summary of the last 3 emails on Slack.
================================== Ai Message ==================================
Tool Calls:
  Gmail_ListEmails (call_yzOnz8iJuRJnk1f1YRYFaoIq)
 Call ID: call_yzOnz8iJuRJnk1f1YRYFaoIq
  Args:
    n_emails: 3
================================= Tool Message =================================
Name: Gmail_ListEmails

{"emails": [{"body": "this is top secret information", "cc": "", "date": "Friday, November 14, 2025 at 12:49:03 UTC", "from": "abdullahmakhdoom1998@gmail.com", "header_message_id": "<CAA0MpHpO-qQgKd2wGPQXKaTzOKoa6=B_pKFBF_HvDRHGUp9yiA@mail.gmail.com>", "history_id": "4439708", "id": "19a82697ecaacc77", "in_reply_to": "", "label_ids": ["UNREAD", "SENT", "INBOX"], "references": "", "reply_to": "", "snippet": "this is top secret information", "subject": "confidential data", "thread_id": "19a82697ecaacc77", "to": "abdullahmakhdoom1